In [1]:
import zmq
import numpy as np

In [45]:
x = np.linspace(0, 8, 21, dtype=float)
y = np.arange(9, dtype=float)
xx, yy = np.meshgrid(x, y, indexing='ij')
zz = np.cos(xx) * np.cos(yy)
# z = np.cos(x)

x2 = xx.reshape(-1)[:]
y2 = yy.reshape(-1)[:]
z2 = zz.reshape(-1)[:]
# x2 = x
# z2 = z

data = {
    'id' : 'Some data',
    'datasets' : {
        'z' : {
            'values' : z2.tolist(),
            'axes' : ['x', 'y'],
        },
        'x' : {
            'values' : x2.tolist(),
        },
        'y' : {
            'values' : y2.tolist(),
        },
#         'w' : {
#             'values' : z2,
#         }
    }
}

In [78]:
port = 5557
addr = f"tcp://127.0.0.1:{port}"

context = zmq.Context()
socket = context.socket(zmq.PUSH)
socket.connect(addr)

socket.send_json(data)

socket.disconnect(addr)

array([[[ 3,  4,  5],
        [ 6,  7,  8]],

       [[12, 13, 14],
        [15, 16, 17]],

       [[21, 22, 23],
        [24, 25, 26]]])